In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.pyplot as plt1 

import statistics
import os

###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import  peak_widths

from scipy.signal import savgol_filter 

###############################################
from bokeh.plotting import figure , show
from pybaselines import whittaker as pl



from bokeh.io import output_notebook
output_notebook()

In [ ]:
def load_data(folder_path):
    # List to store DataFrames for intensity columns
    intensity_dfs = []

    # List to store file names
    file_names = []

    # Get a list of .txt files in the folder
    txt_files = [file_name for file_name in os.listdir(folder_path) if file_name.endswith('.txt')]

    # Sort the .txt files based on their numerical order
    # txt_files.sort(key=lambda x: int(re.search(r'_(\d+)\.txt', x).group(1)))

    # Read the wavelength values from the first file
    first_file_path = os.path.join(folder_path, txt_files[0])
    wavelength_df = pd.read_csv(first_file_path, header=None, delimiter=';', usecols=[0], names=['wavelength'])

    # Loop through each file in ascending order
    for file_name in txt_files:
        file_path = os.path.join(folder_path, file_name)
        
        # Read intensity values from each file into a DataFrame
        intensity_df = pd.read_csv(file_path, header=None, delimiter=';', usecols=[1], names=['intensity'])
        
        # Store intensity DataFrame
        intensity_dfs.append(intensity_df)
        
        # Store file name
        file_names.append(os.path.splitext(file_name)[0])

    # Concatenate intensity DataFrames
    result_df = pd.concat(intensity_dfs, axis=1)

    # Add the wavelength column to the result DataFrame
    result_df = pd.concat([wavelength_df, result_df], axis=1)

    # Rename the columns with file names
    result_df.columns = ['wavelength'] + file_names

    return result_df


In [ ]:
SAMPLE1_Raw_df = load_data('Batch_2/sample_1')
###################################################
SAMPLE2_Raw_df = load_data('Batch_2/sample_2')
###################################################
SAMPLE3_Raw_df = load_data('Batch_2/sample_3')
###################################################
SAMPLE4_Raw_df = load_data('Batch_2/sample_4')
###################################################
SAMPLE5_Raw_df = load_data('Batch_2/sample_5')
###################################################
SAMPLE6_Raw_df = load_data('Batch_2/sample_6')
###################################################
SAMPLE7_Raw_df = load_data('Batch_2/sample_7')
####################################################
SAMPLE8_Raw_df = load_data('Batch_2/sample_8')
####################################################
SAMPLE9_Raw_df = load_data('Batch_3/sample_9')
####################################################
SAMPLE10_Raw_df = load_data('Batch_3/sample_10')
####################################################
SAMPLE11_Raw_df = load_data('Batch_3/sample_11')
####################################################
SAMPLE12_Raw_df = load_data('Batch_3/sample_12')
####################################################
SAMPLE13_Raw_df = load_data('Batch_3/sample_13')
####################################################
SAMPLE14_Raw_df = load_data('Batch_3/sample_14')
####################################################
SAMPLE15_Raw_df = load_data('Batch_3/sample_15')
####################################################
SAMPLE16_Raw_df = load_data('Batch_3/sample_16')
####################################################
SAMPLE17_Raw_df = load_data('Batch_3/sample_17')
####################################################



In [ ]:
import numpy as np

# Assuming you have dataframes named SAMPLE1_Raw_df, SAMPLE2_Raw_df, etc.
# Store them in a dictionary
datafram = {
    1: SAMPLE1_Raw_df,
    2: SAMPLE2_Raw_df,
    3: SAMPLE3_Raw_df,
    4: SAMPLE4_Raw_df,
    5: SAMPLE5_Raw_df,
    6: SAMPLE6_Raw_df,
    7: SAMPLE7_Raw_df,
    8: SAMPLE8_Raw_df,
    9: SAMPLE9_Raw_df,
    10: SAMPLE10_Raw_df,
    11: SAMPLE11_Raw_df,
    12: SAMPLE12_Raw_df,
    13: SAMPLE13_Raw_df,
    14: SAMPLE14_Raw_df,
    15: SAMPLE15_Raw_df,
    16: SAMPLE16_Raw_df,
    17: SAMPLE17_Raw_df,   
    # Add entries for other samples
}

# Define the sample number you want to process
sample_number = 2 # Replace with any sample number (e.g., 3, 4, ..., 17)

# Get the dataframe for the specified sample number
df_sample = datafram[sample_number]

# Construct the column name based on the sample number
column_name = f'sample{sample_number}_6'

# Filter the dataframe to get the noise list
noise_list = df_sample.loc[df_sample['wavelength'] < 215.0, column_name]

# Calculate the RMS intensity noise
RMS_intensity_noise = np.sqrt(np.mean((noise_list)**2))

# Print the RMS intensity noise
print(RMS_intensity_noise)


In [ ]:

# #METHOD  - Scipy

# Threshold = 20 #(Noise Intensity * Threshold) Value for signal filtering
# height_2 = None
# Peak_Prominenace = (Threshold * RMS_intensity_noise) # #Uses S/N ratio , as comapred to other peaks
# minimum_distance_3 = None   #minimum distance between two peaks

# Sample_Peak, info = fp(SAMPLE2_Raw_df.sample2_6,height=height_2,prominence=Peak_Prominenace,distance=minimum_distance_3,width=0.1)


# Selected_Sample_Spectra = SAMPLE2_Raw_df.iloc[Sample_Peak] 

# Sample_Spectra_Peaks = figure(title = 'Peaks in hLIBS Spectra', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
# Sample_Spectra_Peaks.line(SAMPLE2_Raw_df.wavelength,SAMPLE2_Raw_df.sample2_6 , line_width = 2, color ="green" )
# Sample_Spectra_Peaks.circle(Selected_Sample_Spectra.wavelength , Selected_Sample_Spectra.sample2_6 , color = 'red' , size = 8)

# Sample_Spectra_Peaks.width = 1200
# Sample_Spectra_Peaks.height = 600

# print(Selected_Sample_Spectra)
# show(Sample_Spectra_Peaks)


# Define the threshold and other parameters for peak finding
Threshold = 6  # (Noise Intensity * Threshold) Value for signal filtering
height_2 = None
Peak_Prominenace = (Threshold * RMS_intensity_noise)  # Uses S/N ratio, as compared to other peaks
minimum_distance_3 = None  # Minimum distance between two peaks

# Find the peaks
Sample_Peak, info = fp(df_sample[column_name], height=height_2, prominence=Peak_Prominenace, distance=minimum_distance_3, width=0.1)

# Select the sample spectra
Selected_Sample_Spectra = df_sample.iloc[Sample_Peak]

# Create the plot
Sample_Spectra_Peaks = figure(title='Peaks in hLIBS Spectra', x_axis_label='Wavelength', y_axis_label='Intensity')
Sample_Spectra_Peaks.line(df_sample['wavelength'], df_sample[column_name], line_width=2, color="green")
Sample_Spectra_Peaks.circle(Selected_Sample_Spectra['wavelength'], Selected_Sample_Spectra[column_name], color='red', size=5)

Sample_Spectra_Peaks.width = 1200
Sample_Spectra_Peaks.height = 600

# Print the selected sample spectra
print(Selected_Sample_Spectra)

# Show the plot
show(Sample_Spectra_Peaks)



In [ ]:
# Peak Selection and Data Trimming

In [ ]:
# peak = 500.606
# peak_min = (peak - 5) 
# peak_max = (peak + 5) 

#Getting the full wavelength
Wavelength_Min = 200.041162
Wavelength_Max = 963.333130


SAMPLE1_Select_df = SAMPLE1_Raw_df[(SAMPLE1_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE1_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE2_Select_df = SAMPLE2_Raw_df[(SAMPLE2_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE2_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE3_Select_df = SAMPLE3_Raw_df[(SAMPLE3_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE3_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE4_Select_df = SAMPLE4_Raw_df[(SAMPLE4_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE4_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE5_Select_df = SAMPLE5_Raw_df[(SAMPLE5_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE5_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE6_Select_df = SAMPLE6_Raw_df[(SAMPLE6_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE6_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE7_Select_df = SAMPLE7_Raw_df[(SAMPLE7_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE7_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE8_Select_df = SAMPLE8_Raw_df[(SAMPLE8_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE8_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE9_Select_df = SAMPLE9_Raw_df[(SAMPLE9_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE9_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE10_Select_df = SAMPLE10_Raw_df[(SAMPLE10_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE10_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE11_Select_df = SAMPLE11_Raw_df[(SAMPLE11_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE11_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE12_Select_df = SAMPLE12_Raw_df[(SAMPLE12_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE12_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE13_Select_df = SAMPLE13_Raw_df[(SAMPLE13_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE13_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE14_Select_df = SAMPLE14_Raw_df[(SAMPLE14_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE14_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE15_Select_df = SAMPLE15_Raw_df[(SAMPLE15_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE15_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE16_Select_df = SAMPLE16_Raw_df[(SAMPLE16_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE16_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE17_Select_df = SAMPLE17_Raw_df[(SAMPLE17_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE17_Raw_df['wavelength'] <= Wavelength_Max)]




SAMPLE17_Select_df

In [ ]:
Selected_df_Plot = figure(title = 'Selected Data Plot' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Selected_df_Plot.line(SAMPLE1_Select_df.wavelength,SAMPLE1_Select_df.sample1_6 , line_width = 2, color ="blue" )
#####################################################################################################################
Selected_df_Plot.line(SAMPLE2_Select_df.wavelength,SAMPLE2_Select_df.sample2_6 , line_width = 2, color ="orange" )
###############################################################################################################
Selected_df_Plot.line(SAMPLE3_Select_df.wavelength,SAMPLE3_Select_df.sample3_6 , line_width = 2, color ="green")
######################################################################################################################
Selected_df_Plot.line(SAMPLE4_Select_df.wavelength,SAMPLE4_Select_df.sample4_6 , line_width = 2, color ="red")
##############################################################################################################
Selected_df_Plot.line(SAMPLE5_Select_df.wavelength,SAMPLE5_Select_df.sample5_6 , line_width = 2, color ="purple")
###############################################################################################################
Selected_df_Plot.line(SAMPLE6_Select_df.wavelength,SAMPLE6_Select_df.sample6_6 , line_width = 2, color ="brown")
####################################################################################################################
Selected_df_Plot.line(SAMPLE7_Select_df.wavelength,SAMPLE7_Select_df.sample7_6 , line_width = 2, color ="pink")
##################################################################################################################
Selected_df_Plot.line(SAMPLE8_Select_df.wavelength,SAMPLE8_Select_df.sample8_6 , line_width = 2, color ="gray")
###################################################################################################################
Selected_df_Plot.line(SAMPLE9_Select_df.wavelength,SAMPLE9_Select_df.sample9_6 , line_width = 2, color ="olive")
###################################################################################################################
Selected_df_Plot.line(SAMPLE10_Select_df.wavelength,SAMPLE10_Select_df.sample10_6 , line_width = 2, color ="Cyan")
###################################################################################################################
Selected_df_Plot.line(SAMPLE11_Select_df.wavelength,SAMPLE11_Select_df.sample11_6 , line_width = 2, color ="black")
###################################################################################################################
Selected_df_Plot.line(SAMPLE12_Select_df.wavelength,SAMPLE12_Select_df.sample12_6 , line_width = 2, color ="tomato")
###################################################################################################################
Selected_df_Plot.line(SAMPLE13_Select_df.wavelength,SAMPLE13_Select_df.sample13_6 , line_width = 2, color ="steelblue")
###################################################################################################################
Selected_df_Plot.line(SAMPLE14_Select_df.wavelength,SAMPLE14_Select_df.sample14_6 , line_width = 2, color ="limegreen")
###################################################################################################################
Selected_df_Plot.line(SAMPLE15_Select_df.wavelength,SAMPLE15_Select_df.sample15_6 , line_width = 2, color ="deeppink")
###################################################################################################################
Selected_df_Plot.line(SAMPLE16_Select_df.wavelength,SAMPLE16_Select_df.sample16_6 , line_width = 2, color ="gold")
###################################################################################################################
Selected_df_Plot.line(SAMPLE17_Select_df.wavelength,SAMPLE17_Select_df.sample17_6 , line_width = 2, color ="indigo")
###################################################################################################################

Selected_df_Plot.width = 900
Selected_df_Plot.height = 500
show(Selected_df_Plot)

# Data Preprocessing of the Spectra

In [ ]:
def baseline_correction(df):
    """
    Perform baseline correction on the intensity columns of the input DataFrame and create a new DataFrame with corrected values.
    
    Parameters:
        df (DataFrame): Input DataFrame containing the wavelength and intensity columns.
        
    Returns:
        DataFrame: New DataFrame with baseline-corrected intensity columns and the same wavelength column as the input DataFrame.
    """
    # Copy the 'wavelength' column from the input DataFrame
    new_df = pd.DataFrame({'wavelength': df['wavelength']})
    
    # Perform baseline correction for each intensity column and add them to the new DataFrame
    for col in df.columns[1:]:  # Exclude the 'wavelength' column
        baseline, _ = pl.airpls(df[col])
        corrected_values = df[col] - baseline
        new_df[col] = corrected_values
    
    return new_df

In [ ]:
SAMPLE1_BaselineCorrected_df = baseline_correction(SAMPLE1_Raw_df)
SAMPLE2_BaselineCorrected_df = baseline_correction(SAMPLE2_Raw_df)
SAMPLE3_BaselineCorrected_df = baseline_correction(SAMPLE3_Raw_df)
SAMPLE4_BaselineCorrected_df = baseline_correction(SAMPLE4_Raw_df)
SAMPLE5_BaselineCorrected_df = baseline_correction(SAMPLE5_Raw_df)
SAMPLE6_BaselineCorrected_df = baseline_correction(SAMPLE6_Raw_df)
SAMPLE7_BaselineCorrected_df = baseline_correction(SAMPLE7_Raw_df)
SAMPLE8_BaselineCorrected_df = baseline_correction(SAMPLE8_Raw_df)
SAMPLE9_BaselineCorrected_df = baseline_correction(SAMPLE9_Raw_df)
SAMPLE10_BaselineCorrected_df = baseline_correction(SAMPLE10_Raw_df)
SAMPLE11_BaselineCorrected_df = baseline_correction(SAMPLE11_Raw_df)
SAMPLE12_BaselineCorrected_df = baseline_correction(SAMPLE12_Raw_df)
SAMPLE13_BaselineCorrected_df = baseline_correction(SAMPLE13_Raw_df)
SAMPLE14_BaselineCorrected_df = baseline_correction(SAMPLE14_Raw_df)
SAMPLE15_BaselineCorrected_df = baseline_correction(SAMPLE15_Raw_df)
SAMPLE16_BaselineCorrected_df = baseline_correction(SAMPLE16_Raw_df)
SAMPLE17_BaselineCorrected_df = baseline_correction(SAMPLE17_Raw_df)

SAMPLE1_BaselineCorrected_df

In [ ]:
def standard_normal_variate_normalization(df):
    # Select columns containing intensities
    intensity_cols = df.columns[1:]  #  columns 2 to 13 are intensities and are stored in intensity_cols

    # SVN normalization
    for col in intensity_cols:
        mean_intensity = df[col].mean()
        std_intensity = df[col].std()
        df[col] = (df[col] - mean_intensity) / std_intensity

    return df

In [ ]:
def total_intensity_normalization(df):
    # Selecting only the intensity columns (assuming the first column is wavelength)
    intensities = df.iloc[:, 1:]  

    # Initialize an empty DataFrame to store normalized intensities
    normalized_intensities = pd.DataFrame()

    # Iterate over each column (each spectrum)
    for column in intensities.columns:
        spectrum = intensities[column]
        total_intensity = spectrum.sum()
        
        # Normalize the spectrum by dividing each intensity value by the total intensity
        normalized_spectrum = spectrum / total_intensity
        
        # Append the normalized spectrum to the DataFrame of normalized intensities
        normalized_intensities[column] = normalized_spectrum

    # Combine the wavelength column with normalized intensities
    normalized_df = pd.concat([df.iloc[:, 0], normalized_intensities], axis=1)

    return normalized_df

In [ ]:
def unit_norm_normalization(df):
    # Selecting only the intensity columns (assuming the first column is wavelength)
    intensities_1 = df.iloc[:, 1:]  
    
    # Initialize an empty DataFrame to store normalized intensities
    normalized_intensities_1 = pd.DataFrame()

    # Iterate over each column (each spectrum)
    for column_1 in intensities_1.columns:
        spectrum_1 = intensities_1[column_1]
        
        # Calculate the norm of the spectrum using a simple equation
        spectrum_norm = np.sqrt(np.sum(spectrum_1 ** 2))
        
        # Normalize the spectrum by dividing each intensity value by its norm
        normalized_spectrum_1 = spectrum_1 / spectrum_norm
        
        # Assign the normalized spectrum to the DataFrame of normalized intensities
        normalized_intensities_1[column_1] = normalized_spectrum_1

    # Combine the wavelength column with normalized intensities
    normalized_df_1 = pd.concat([df.iloc[:, 0], normalized_intensities_1], axis=1)
    
    return normalized_df_1

In [ ]:
def max_intensity_normalization(df):
    # Selecting only the intensity columns (assuming the first column is wavelength)
    intensities_2 = df.iloc[:, 1:]  

    # Initialize an empty DataFrame to store normalized intensities
    normalized_intensities_2 = pd.DataFrame()

    # Iterate over each column (each spectrum)
    for column_2 in intensities_2.columns:
        spectrum_2 = intensities_2[column_2]
        
        # Find the maximum intensity value in the spectrum
        max_intensity = spectrum_2.max()
        
        # Normalize the spectrum by dividing each intensity value by the maximum intensity
        normalized_spectrum_2 = spectrum_2 / max_intensity
        
        # Assign the normalized spectrum to the DataFrame of normalized intensities
        normalized_intensities_2[column_2] = normalized_spectrum_2

    # Combine the wavelength column with normalized intensities
    normalized_df_2 = pd.concat([df.iloc[:, 0], normalized_intensities_2], axis=1)

    return normalized_df_2

In [ ]:
SAMPLE1_Normalized_df = unit_norm_normalization(SAMPLE1_BaselineCorrected_df)
SAMPLE2_Normalized_df = unit_norm_normalization(SAMPLE2_BaselineCorrected_df)
SAMPLE3_Normalized_df = unit_norm_normalization(SAMPLE3_BaselineCorrected_df)
SAMPLE4_Normalized_df = unit_norm_normalization(SAMPLE4_BaselineCorrected_df)
SAMPLE5_Normalized_df = unit_norm_normalization(SAMPLE5_BaselineCorrected_df)
SAMPLE6_Normalized_df = unit_norm_normalization(SAMPLE6_BaselineCorrected_df)
SAMPLE7_Normalized_df = unit_norm_normalization(SAMPLE7_BaselineCorrected_df)
SAMPLE8_Normalized_df = unit_norm_normalization(SAMPLE8_BaselineCorrected_df)
SAMPLE9_Normalized_df = unit_norm_normalization(SAMPLE9_BaselineCorrected_df)
SAMPLE10_Normalized_df = unit_norm_normalization(SAMPLE10_BaselineCorrected_df)
SAMPLE11_Normalized_df = unit_norm_normalization(SAMPLE11_BaselineCorrected_df)
SAMPLE12_Normalized_df = unit_norm_normalization(SAMPLE12_BaselineCorrected_df)
SAMPLE13_Normalized_df = unit_norm_normalization(SAMPLE13_BaselineCorrected_df)
SAMPLE14_Normalized_df = unit_norm_normalization(SAMPLE14_BaselineCorrected_df)
SAMPLE15_Normalized_df = unit_norm_normalization(SAMPLE15_BaselineCorrected_df)
SAMPLE16_Normalized_df = unit_norm_normalization(SAMPLE16_BaselineCorrected_df)
SAMPLE17_Normalized_df = unit_norm_normalization(SAMPLE17_BaselineCorrected_df)

SAMPLE1_Normalized_df

In [ ]:
def average_intensity(df, intensity_name):
    # Select intensity columns (from column 2 to column 13)
    intensity_columns = df.columns[1:]

    # Calculate the mean of intensity columns
    averaged_intensity = df[intensity_columns].mean(axis=1)

    # Create a new DataFrame with wavelength and averaged intensity
    averaged_df = pd.DataFrame({'wavelength': df['wavelength'], intensity_name: averaged_intensity})

    return averaged_df

In [ ]:
SAMPLE1_Averaged_df = average_intensity(df = SAMPLE1_Normalized_df , intensity_name= 'sample1')
SAMPLE2_Averaged_df = average_intensity(df = SAMPLE2_Normalized_df , intensity_name= 'sample2')
SAMPLE3_Averaged_df = average_intensity(df = SAMPLE3_Normalized_df , intensity_name= 'sample3')
SAMPLE4_Averaged_df = average_intensity(df = SAMPLE4_Normalized_df , intensity_name= 'sample4')
SAMPLE5_Averaged_df = average_intensity(df = SAMPLE5_Normalized_df , intensity_name= 'sample5')
SAMPLE6_Averaged_df = average_intensity(df = SAMPLE6_Normalized_df , intensity_name= 'sample6')
SAMPLE7_Averaged_df = average_intensity(df = SAMPLE7_Normalized_df , intensity_name= 'sample7')
SAMPLE8_Averaged_df = average_intensity(df = SAMPLE8_Normalized_df , intensity_name= 'sample8')
SAMPLE9_Averaged_df = average_intensity(df = SAMPLE9_Normalized_df , intensity_name= 'sample9')
SAMPLE10_Averaged_df = average_intensity(df = SAMPLE10_Normalized_df , intensity_name= 'sample10')
SAMPLE11_Averaged_df = average_intensity(df = SAMPLE11_Normalized_df , intensity_name= 'sample11')
SAMPLE12_Averaged_df = average_intensity(df = SAMPLE12_Normalized_df , intensity_name= 'sample12')
SAMPLE13_Averaged_df = average_intensity(df = SAMPLE13_Normalized_df , intensity_name= 'sample13')
SAMPLE14_Averaged_df = average_intensity(df = SAMPLE14_Normalized_df , intensity_name= 'sample14')
SAMPLE15_Averaged_df = average_intensity(df = SAMPLE15_Normalized_df , intensity_name= 'sample15')
SAMPLE16_Averaged_df = average_intensity(df = SAMPLE16_Normalized_df , intensity_name= 'sample16')
SAMPLE17_Averaged_df = average_intensity(df = SAMPLE17_Normalized_df , intensity_name= 'sample17')

print(SAMPLE1_Averaged_df)

In [ ]:

# List of DataFrame names
data_frames = [SAMPLE1_Averaged_df, SAMPLE2_Averaged_df, SAMPLE3_Averaged_df, 
               SAMPLE4_Averaged_df, SAMPLE5_Averaged_df, SAMPLE6_Averaged_df, 
               SAMPLE7_Averaged_df, SAMPLE8_Averaged_df ,SAMPLE9_Averaged_df,
               SAMPLE10_Averaged_df,SAMPLE11_Averaged_df,SAMPLE12_Averaged_df,
               SAMPLE13_Averaged_df,SAMPLE14_Averaged_df,SAMPLE15_Averaged_df,
               SAMPLE16_Averaged_df,SAMPLE17_Averaged_df]

# Initialize the new DataFrame with the 'wavelength' column from the first DataFrame
new_df = data_frames[0][['wavelength']].copy()

# Add 'sample' columns from each DataFrame
for i, df in enumerate(data_frames):
    sample_column = f'sample{i+1}'
    new_df[sample_column] = df[sample_column]

# Display the new DataFrame
print(new_df)

# SELECTION OF PEAKS , TO DETERMINE THE PEAK START AND STOP 

# Wavelengths for Detected Peak(red circles above)

In [ ]:
Sample_Spectra_List = []

for iteam in Selected_Sample_Spectra.wavelength:
    iteam = float(iteam)
    Sample_Spectra_List.append(iteam)
    
print(Sample_Spectra_List)


In [ ]:
Selected_Sample_Spectra_min_list = []

Selected_Sample_Spectra_left = SAMPLE1_Averaged_df.iloc[info['left_ips']] 
Selected_Sample_Spectra_min_list = (Selected_Sample_Spectra_left['wavelength']).tolist()

print (Selected_Sample_Spectra_min_list)



In [ ]:
Selected_Sample_Spectra_max_list = []


Selected_Sample_Spectra_right = SAMPLE1_Averaged_df.iloc[info['right_ips']] 
Selected_Sample_Spectra_max_list = (Selected_Sample_Spectra_right['wavelength']).tolist()

print (Selected_Sample_Spectra_max_list)

# COMBINED LIST

In [ ]:
wavelength_min_max_list = [[a, b] for a, b in zip(Selected_Sample_Spectra_min_list, Selected_Sample_Spectra_max_list)]

print(wavelength_min_max_list)



In [ ]:

# Create a list to store the maximum peaks for each wavelength range
max_peaks_all_ranges = []

# Loop through each wavelength range
for wavelength_min, wavelength_max in wavelength_min_max_list:
    # Filter the DataFrame for the current wavelength range
    filtered_df = new_df[(new_df["wavelength"] >= wavelength_min) & (new_df["wavelength"] <= wavelength_max)]
    #print(filtered_df)
    
    
    # Find the maximum value for each sample column within the filtered DataFrame
    max_peaks = []
    for sample in filtered_df.columns[1:]:  # Skip the 'wavelength' column
        max_value = filtered_df[sample].max()
        min_value = filtered_df[sample].min()
        max_peaks.append(max_value)
    
    # Append the list of maximum peaks for this range to the main list
    max_peaks_all_ranges.append(max_peaks)


# Print the results
#for i, max_peaks in enumerate(max_peaks_all_ranges, start=1):
    #print(f"Range {i} max peaks: {max_peaks}")

# LIST FOR NIST ELEMENT PEAKS

In [ ]:
ele = 'Mo'
# NISTDB_Reference_Spectra = pd.read_csv(r"NIST_3/" + ele +".csv" , encoding = 'windows - 1252' , usecols=(3,4,5,6,7,8))
# Reference_Spectra_List = (NISTDB_Reference_Spectra['obs_wl_air(nm)']).tolist()

################################################################################################

New_Reference_Spectra = pd.read_csv(r"NIST_2/" + ele +".csv" , encoding = 'windows - 1252' , usecols=(0,1))


Reference_Spectra_List = (New_Reference_Spectra['Wavelength'] / 10).tolist()
print(Reference_Spectra_List)


In [ ]:
Reference_Spectra_Min_Max = []
Reference_Spectra_Min_Max = [[num - 0.025, num + 0.025] for num in Reference_Spectra_List]     
    
print(Reference_Spectra_Min_Max)

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd

# Provided data (as an example, add more sublists as needed)
data = max_peaks_all_ranges

elements = {
    'Element': ['C'  , 'Mn', 'Si', 'Al', 'Cr', 'Cu', 'N', 'Ni' , 'Nb' ,'Mo', 'Ti'],
    'SAMPLE1': [0.15 , 1.9, 0.3, 0.04, 0.41, 0.02, 0.004, 0.030, 0.006, 0.101, 0.044],
    'SAMPLE2': [0.15 , 1.8, 0.3, 0.04, 0.42, 0.02, 0.004, 0.037, 0.006, 0.105, 0.037],
    'SAMPLE3': [0.16 , 2.3, 0.5, 0.69, 0.70, 0.03, 0.003, 0.048, 0.032, 0.023, 0.025],
    'SAMPLE4': [0.23 , 1.2, 0.2, 0.03, 0.14, 0.02, 0.004, 0.020, 0.001, 0.000, 0.034],
    'SAMPLE5': [0.21 , 1.9, 1.8, 0.04, 0.05, 0.03, 0.003, 0.037, 0.056, 0.006, 0.014],
    'SAMPLE6': [0.07 , 2.3, 0.2, 0.04, 0.60, 0.02, 0.005, 0.040, 0.001, 0.020, 0.051],
    'SAMPLE7': [0.003, 0.1, 0.0, 0.02, 0.03, 0.01, 0.002, 0.020, 0.001, 0.005, 0.068],
    'SAMPLE8': [0.06 , 1.1, 0.4, 0.04, 0.03, 0.01, 0.004, 0.010, 0.046, 0.002, 0.028],
}

# Extract 'Mn' values for samples 1 to 8
mn_values = [elements[f'SAMPLE{i}'][9] for i in range(1, 9)]

results = []

# Loop through each sublist in data and perform linear regression
for idx, sublist in enumerate(data):
    X = np.array(sublist[:8]).reshape(-1, 1)  # Take only the first 8 values corresponding to samples 1 to 8
    y = np.array(mn_values)
    
    # Create and train the model
    model = LinearRegression()
    model.fit(X, y)
    
    # Predict the values
    #y_pred = model.predict(X)
    
    # Calculate R² and RMSE
    r2 = model.score(X,y)
    # Calculate Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mean_squared_error(X,y))
    slope = model.coef_
    intercept = model.intercept_

    for cell in Reference_Spectra_Min_Max: 
        if Sample_Spectra_List[idx] >= cell[0] and Sample_Spectra_List[idx] <= cell[1]:
            results.append({'wavelength': Sample_Spectra_List[idx], 'wavelength_min': Selected_Sample_Spectra_min_list[idx] , 'wavelength_max': Selected_Sample_Spectra_max_list[idx], 'R2': r2, 'RMSE': rmse ,'slope': slope , 'intercept': intercept , 'Index' : idx } )

# # Sort the results by R² in descending order
# sorted_results = sorted(results, key=lambda x: x['R2'], reverse=True)

# Filter the results where slope is positive
filtered_results = [result for result in results if result['slope'] > 0]

# Sort the filtered results by R² in descending order
sorted_filtered_results = sorted(filtered_results, key=lambda x: x['R2'], reverse=True)

# Display the results
df = pd.DataFrame(sorted_filtered_results[:50])
print(df)

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd

# Provided data (as an example, add more sublists as needed)
data_1 = max_peaks_all_ranges


elements_1 = {
    'Element':  [   'C', 'Si',  'Mn',    'P',    'S',  'Cr',  'Mo',  'Ni',  'Al',  'Co',  'Cu',   'Nb',   'Ti',    'V',   'Sn',   'W',   'Zr',   'Pb',    'B'],
    'SAMPLE9':  [0.1222, 0.13, 0.571, 0.0257, 0.0075, 0.550, 0.000, 0.000, 0.280, 0.000, 0.036, 0.0191, 0.0353, 0.0071, 0.0000, 0.000, 0.0065, 0.0000, 0.0016],
    'SAMPLE10': [0.0847, 0.10, 0.489, 0.0156, 0.0140, 0.020, 0.007, 0.000, 0.413, 0.000, 0.101, 0.0000, 0.0086, 0.0038, 0.0000, 0.016, 0.0069, 0.0000, 0.0015],
    'SAMPLE11': [0.2990, 0.32, 0.622, 0.0205, 0.0079, 2.150, 0.126, 0.000, 3.430, 0.000, 0.049, 0.0337, 0.0502, 0.0246, 0.0000, 0.000, 0.0078, 0.0000, 0.0027],
    'SAMPLE12': [0.0032, 0.00, 0.000, 0.0049, 0.0058, 0.000, 0.007, 0.000, 0.000, 0.000, 0.006, 0.0000, 0.0000, 0.0000, 0.000, 0.021, 0.0055, 0.0000, 0.0000],
    'SAMPLE13': [0.2734, 0.09, 0.347, 0.0225, 0.0147, 0.021, 0.008, 0.047, 0.000, 0.000, 0.044, 0.0021, 0.0000, 0.0029, 0.0000, 0.000, 0.0071, 0.0000, 0.0009],
    'SAMPLE14': [0.8027, 0.67, 0.428, 0.0402, 0.0240, 0.591, 0.010, 0.054, 0.775, 0.000, 0.220, 0.0169, 0.0181, 0.0087, 0.0000, 0.000, 0.0091, 0.0000, 0.0019],
    'SAMPLE15': [0.6353, 0.58, 0.473, 0.0415, 0.0238, 0.184, 0.005, 0.075, 1.995, 0.000, 0.430, 0.0136, 0.0158, 0.0061, 0.0120, 0.000, 0.0121, 0.0000, 0.0023],
    'SAMPLE16': [0.6632, 0.33, 0.515, 0.0323, 0.0176, 0.194, 0.021, 0.077, 0.029, 0.000, 1.585, 0.0000, 0.0067, 0.0000, 0.0150, 0.000, 0.0000, 0.0000, 0.0018],
    'SAMPLE17': [0.0943, 0.00, 0.208, 0.0250, 0.0130, 0.007, 0.007, 0.023, 0.000, 0.000, 0.067, 0.0030, 0.0000, 0.0023, 0.0000, 0.000, 0.0071, 0.0000, 0.0005],
}



# Extract 'Mn' values for samples 9 to 18
mn_values_1 = [elements_1[f'SAMPLE{i}'][6] for i in range(9, 18)]

results_1 = []

# Loop through each sublist in data and perform linear regression
for idx_1, sublist in enumerate(data_1):
    X_1 = np.array(sublist[8:]).reshape(-1, 1)  # Take only the values corresponding to samples 9 to 17
    y_1 = np.array(mn_values_1)
    
    # Create and train the model
    model_1 = LinearRegression()
    model_1.fit(X_1, y_1)
    
    # Predict the values
    #y_pred = model.predict(X)
    
   # Calculate R² and RMSE
    r2_1 = model_1.score(X_1,y_1)
    # Calculate Root Mean Squared Error (RMSE)
    rmse_1 = np.sqrt(mean_squared_error(X_1,y_1))
    slope_1 = model_1.coef_
    intercept_1 = model_1.intercept_
    for cell_1 in Reference_Spectra_Min_Max: 
        if Sample_Spectra_List[idx_1] >= cell_1[0] and Sample_Spectra_List[idx_1] <= cell_1[1]:
                results_1.append({'wavelength': Sample_Spectra_List[idx_1],'wavelength_min': Selected_Sample_Spectra_min_list[idx_1] , 'wavelength_max': Selected_Sample_Spectra_max_list[idx_1], 'R2': r2_1, 'RMSE': rmse_1 , 'slope': slope_1 , 'intercept': intercept_1, 'Index' : idx_1 } )


# Filter the results where slope is positive
filtered_results_1 = [result for result in results_1 if result['slope'] > 0]

# Sort the filtered results by R² in descending order
sorted_filtered_results_1 = sorted(filtered_results_1, key=lambda x: x['R2'], reverse=True)

# Display the top 50 results
df_1 = pd.DataFrame(sorted_filtered_results_1[:50])
print(df_1)
